In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1212231,2021-04-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1212232,2021-04-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1212233,2021-04-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1212234,2021-04-27,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
32411,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32413,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32415,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32417,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32419,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
746299,2021-04-24,Arkansas,Arkansas,5001,2092,36.00,5000,Arkansas,AR,Arkansas,State,3017804
746301,2021-04-25,Arkansas,Arkansas,5001,2092,36.00,5000,Arkansas,AR,Arkansas,State,3017804
746303,2021-04-26,Arkansas,Arkansas,5001,2092,36.00,5000,Arkansas,AR,Arkansas,State,3017804
746305,2021-04-27,Arkansas,Arkansas,5001,2092,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1212231,2021-04-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1212232,2021-04-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1212233,2021-04-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1212234,2021-04-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-28') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
463,2021-04-28,Snohomish,Washington,35643,565.00,53061,WA,County,822083,4335.69,68.73
924,2021-04-28,Cook,Illinois,531906,10482.00,17031,IL,County,5150233,10327.80,203.52
1384,2021-04-28,Orange,California,269949,4944.00,6059,CA,County,3175692,8500.48,155.68
1843,2021-04-28,Maricopa,Arizona,535862,9847.00,4013,AZ,County,4485414,11946.77,219.53
2302,2021-04-28,Los Angeles,California,1233036,23831.00,6037,CA,County,10039107,12282.33,237.38
...,...,...,...,...,...,...,...,...,...,...,...
1211567,2021-04-28,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08
1211765,2021-04-28,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1211932,2021-04-28,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1212095,2021-04-28,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
463,2021-04-28,Snohomish,Washington,35643,565.00,53061,WA,County,822083,4335.69,68.73,68.73,4335.69
924,2021-04-28,Cook,Illinois,531906,10482.00,17031,IL,County,5150233,10327.80,203.52,203.52,10327.80
1384,2021-04-28,Orange,California,269949,4944.00,6059,CA,County,3175692,8500.48,155.68,155.68,8500.48
1843,2021-04-28,Maricopa,Arizona,535862,9847.00,4013,AZ,County,4485414,11946.77,219.53,219.53,11946.77
2302,2021-04-28,Los Angeles,California,1233036,23831.00,6037,CA,County,10039107,12282.33,237.38,237.38,12282.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211567,2021-04-28,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08,75.08,1951.95
1211765,2021-04-28,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1211932,2021-04-28,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1212095,2021-04-28,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
780360,2021-04-28,Hale,Alabama,2203,75.00,1065,AL,County,14651,15036.52,511.91,511.91,15036.52,1
849405,2021-04-28,Clarke,Alabama,3500,60.00,1025,AL,County,23622,14816.70,254.00,254.00,14816.70,2
610581,2021-04-28,Lowndes,Alabama,1383,53.00,1085,AL,County,9726,14219.62,544.93,544.93,14219.62,3
454296,2021-04-28,Franklin,Alabama,4228,82.00,1059,AL,County,31362,13481.28,261.46,261.46,13481.28,4
541387,2021-04-28,Etowah,Alabama,13753,352.00,1055,AL,County,102268,13448.00,344.19,344.19,13448.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149306,2021-04-28,Platte,Wyoming,622,11.00,56031,WY,County,8393,7410.94,131.06,131.06,7410.94,19
744722,2021-04-28,Converse,Wyoming,991,17.00,56009,WY,County,13822,7169.73,122.99,122.99,7169.73,20
925847,2021-04-28,Lincoln,Wyoming,1393,12.00,56023,WY,County,19830,7024.71,60.51,60.51,7024.71,21
1030423,2021-04-28,Niobrara,Wyoming,152,2.00,56027,WY,County,2356,6451.61,84.89,84.89,6451.61,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
610581,2021-04-28,Lowndes,Alabama,1383,53.00,1085,AL,County,9726,14219.62,544.93,544.93,14219.62,3,1
780360,2021-04-28,Hale,Alabama,2203,75.00,1065,AL,County,14651,15036.52,511.91,511.91,15036.52,1,2
259692,2021-04-28,Walker,Alabama,7190,278.00,1127,AL,County,63521,11319.09,437.65,437.65,11319.09,28,3
539787,2021-04-28,Clay,Alabama,1525,56.00,1027,AL,County,13235,11522.48,423.12,423.12,11522.48,23,4
658458,2021-04-28,Greene,Alabama,921,34.00,1063,AL,County,8111,11354.95,419.18,419.18,11354.95,26,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656865,2021-04-28,Hot Springs,Wyoming,367,3.00,56017,WY,County,4413,8316.34,67.98,67.98,8316.34,17,19
925847,2021-04-28,Lincoln,Wyoming,1393,12.00,56023,WY,County,19830,7024.71,60.51,60.51,7024.71,21,20
902778,2021-04-28,Uinta,Wyoming,2180,12.00,56041,WY,County,20226,10778.21,59.33,59.33,10778.21,5,21
258474,2021-04-28,Teton,Wyoming,3729,9.00,56039,WY,County,23464,15892.43,38.36,38.36,15892.43,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.73,4335.69,19,23
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.73,4335.69,19,23
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.73,4335.69,19,23
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.73,4335.69,19,23
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.73,4335.69,19,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209389,2021-04-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1209390,2021-04-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1209391,2021-04-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1209392,2021-04-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
777124,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15036.52,2,1,1.00
777125,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15036.52,2,1,0.00
777126,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15036.52,2,1,0.00
777127,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15036.52,2,1,0.00
777128,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,511.91,15036.52,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941331,2021-04-24,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5656.65,6,23,0.00
941332,2021-04-25,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5656.65,6,23,0.00
941333,2021-04-26,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,1.00
941334,2021-04-27,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
608135,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14219.62,1,3,1.00,0.00
608136,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14219.62,1,3,0.00,0.00
608137,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14219.62,1,3,0.00,0.00
608138,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14219.62,1,3,0.00,0.00
608139,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14219.62,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606135,2021-04-24,Albany,Wyoming,4184,11.00,56001,WY,County,38880,10761.32,28.29,28.29,10851.34,23,4,0.00,0.00
606136,2021-04-25,Albany,Wyoming,4184,11.00,56001,WY,County,38880,10761.32,28.29,28.29,10851.34,23,4,0.00,0.00
606137,2021-04-26,Albany,Wyoming,4198,11.00,56001,WY,County,38880,10797.33,28.29,28.29,10851.34,23,4,14.00,0.00
606138,2021-04-27,Albany,Wyoming,4206,11.00,56001,WY,County,38880,10817.90,28.29,28.29,10851.34,23,4,8.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-28') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
310004,2021-04-28,Imperial,California,28149,720.00,6025,CA,County,181215,15533.48,397.32,397.32,15533.48,1,2,1.00,1.00
2302,2021-04-28,Los Angeles,California,1233036,23831.00,6037,CA,County,10039107,12282.33,237.38,237.38,12282.33,2,5,294.00,29.00
542939,2021-04-28,Inyo,California,1421,38.00,6027,CA,County,18039,7877.38,210.65,210.65,7877.38,3,24,25.00,2.00
154370,2021-04-28,San Bernardino,California,295752,4575.00,6071,CA,County,2180085,13566.08,209.85,209.85,13566.08,4,4,87.00,3.00
69033,2021-04-28,Stanislaus,California,61150,1042.00,6099,CA,County,550660,11104.86,189.23,189.23,11104.86,5,9,55.00,2.00
31992,2021-04-28,Riverside,California,298262,4564.00,6065,CA,County,2470546,12072.72,184.74,184.74,12072.72,6,6,3.00,1.00
85590,2021-04-28,Tulare,California,49585,834.00,6107,CA,County,466195,10636.11,178.90,178.90,10636.11,7,10,38.00,0.00
54925,2021-04-28,San Joaquin,California,72345,1346.00,6077,CA,County,762148,9492.25,176.61,176.61,9492.25,8,19,64.00,0.00
31156,2021-04-28,Fresno,California,101138,1663.00,6019,CA,County,999101,10122.90,166.45,166.45,10122.90,9,14,39.00,1.00
492029,2021-04-28,Merced,California,31529,454.00,6047,CA,County,277680,11354.44,163.50,163.50,11354.44,10,8,40.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1148857,2021-04-28,Lassen,California,5685,24.00,6035,CA,County,30573,18594.84,78.50,78.50,18594.84,34,1,0.00,23.00
310004,2021-04-28,Imperial,California,28149,720.00,6025,CA,County,181215,15533.48,397.32,397.32,15533.48,1,2,1.00,1.00
658400,2021-04-28,Kings,California,22859,246.00,6031,CA,County,152940,14946.38,160.85,160.85,14946.38,11,3,9.00,0.00
154370,2021-04-28,San Bernardino,California,295752,4575.00,6071,CA,County,2180085,13566.08,209.85,209.85,13566.08,4,4,87.00,3.00
2302,2021-04-28,Los Angeles,California,1233036,23831.00,6037,CA,County,10039107,12282.33,237.38,237.38,12282.33,2,5,294.00,29.00
31992,2021-04-28,Riverside,California,298262,4564.00,6065,CA,County,2470546,12072.72,184.74,184.74,12072.72,6,6,3.00,1.00
194085,2021-04-28,Kern,California,108456,1348.00,6029,CA,County,900202,12047.96,149.74,149.74,12047.96,14,7,38.00,3.00
492029,2021-04-28,Merced,California,31529,454.00,6047,CA,County,277680,11354.44,163.50,163.50,11354.44,10,8,40.00,0.00
69033,2021-04-28,Stanislaus,California,61150,1042.00,6099,CA,County,550660,11104.86,189.23,189.23,11104.86,5,9,55.00,2.00
85590,2021-04-28,Tulare,California,49585,834.00,6107,CA,County,466195,10636.11,178.90,178.90,10636.11,7,10,38.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
310004,2021-04-28,Imperial,California,28149,720.00,6025,CA,County,181215,15533.48,397.32,397.32,15533.48,1,2,1.00,1.00
2302,2021-04-28,Los Angeles,California,1233036,23831.00,6037,CA,County,10039107,12282.33,237.38,237.38,12282.33,2,5,294.00,29.00
542939,2021-04-28,Inyo,California,1421,38.00,6027,CA,County,18039,7877.38,210.65,210.65,7877.38,3,24,25.00,2.00
154370,2021-04-28,San Bernardino,California,295752,4575.00,6071,CA,County,2180085,13566.08,209.85,209.85,13566.08,4,4,87.00,3.00
69033,2021-04-28,Stanislaus,California,61150,1042.00,6099,CA,County,550660,11104.86,189.23,189.23,11104.86,5,9,55.00,2.00
31992,2021-04-28,Riverside,California,298262,4564.00,6065,CA,County,2470546,12072.72,184.74,184.74,12072.72,6,6,3.00,1.00
85590,2021-04-28,Tulare,California,49585,834.00,6107,CA,County,466195,10636.11,178.90,178.90,10636.11,7,10,38.00,0.00
54925,2021-04-28,San Joaquin,California,72345,1346.00,6077,CA,County,762148,9492.25,176.61,176.61,9492.25,8,19,64.00,0.00
31156,2021-04-28,Fresno,California,101138,1663.00,6019,CA,County,999101,10122.90,166.45,166.45,10122.90,9,14,39.00,1.00
492029,2021-04-28,Merced,California,31529,454.00,6047,CA,County,277680,11354.44,163.50,163.50,11354.44,10,8,40.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,397.32,15533.48,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,397.32,15533.48,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,397.32,15533.48,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,397.32,15533.48,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,397.32,15533.48,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4062,04/24/21,Lassen,5685,1.00,18594.84,3.27,78.50,18594.84,34,1,0.00,0.00
4063,04/25/21,Lassen,5685,1.00,18594.84,3.27,78.50,18594.84,34,1,0.00,0.00
4064,04/26/21,Lassen,5685,1.00,18594.84,3.27,78.50,18594.84,34,1,0.00,0.00
4065,04/27/21,Lassen,5685,1.00,18594.84,3.27,78.50,18594.84,34,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,397.32,15533.48,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,397.32,15533.48,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,397.32,15533.48,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,397.32,15533.48,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,397.32,15533.48,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4148,04/24/21,Merced,31400,454.00,11307.98,163.50,163.50,11354.44,10,8,0.00,0.00
4149,04/25/21,Merced,31400,454.00,11307.98,163.50,163.50,11354.44,10,8,0.00,0.00
4150,04/26/21,Merced,31452,454.00,11326.71,163.50,163.50,11354.44,10,8,52.00,0.00
4151,04/27/21,Merced,31489,454.00,11340.03,163.50,163.50,11354.44,10,8,37.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)